In [2]:
import torch
from torch import nn, autograd
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd, numpy as np, matplotlib, matplotlib.pyplot as plt
from PIL import Image
import time
import os
from skimage import io, color

import torchvision
from torchvision import transforms, models

In [39]:
train_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([256,256]),
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

validation_transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize([256,256]),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
class BoneAgeDataset_TL(Dataset):

    def __init__(self, csv_file, root_dir, transform=None):

        self.data_frame = pd.read_csv(csv_file)
        self.root_dir = root_dir
        self.transform = transform

    def __len__(self):
        return len(self.data_frame)

    def __getitem__(self, idx):
        img_name = os.path.join(self.root_dir,
                                self.data_frame.iloc[idx, 0])
        
        image = io.imread(img_name)
        if len(image.shape) > 2 and image.shape[2] == 4:
            image = image[:,:,0]
            
        # replicate the image into 3 RGB channels
        image=np.repeat(image[None,...],3,axis=0)
            
        image_age = self.data_frame.iloc[idx, 1]

        if self.transform:
            image = self.transform(image)
            
        sample = {'x': image, 'y': image_age}

        return sample


In [40]:
BATCH_SIZE = 16

BoneAge_TrainData = BoneAgeDataset_TL(csv_file='data/random_train_boneage.csv',
                                    root_dir='/beegfs/ga4493/projects/team_G/boneage-training-dataset_final', transform=train_transform)
train_loader = DataLoader(BoneAge_TrainData, batch_size=BATCH_SIZE,
                        shuffle=True, num_workers=0)

BoneAge_ValidationData = BoneAgeDataset_TL(csv_file='data/random_validation_boneage.csv',
                                    root_dir='/beegfs/ga4493/projects/team_G/boneage-training-dataset_final', transform=validation_transform)
validation_loader = DataLoader(BoneAge_ValidationData, batch_size=BATCH_SIZE,
                        shuffle=False, num_workers=0)

dataset_sizes = {'train': len(BoneAge_TrainData), 'val': len(BoneAge_ValidationData)}


In [53]:
def train_model(model, criterion, optimizer, num_epochs=40, trainVal=['train','val'],verbose=True):
    acc_dict = {'train':[],'val':[]}
    loss_dict = {'train':[],'val':[]}
    best_loss = 100
    since = time.time()
    for epoch in range(num_epochs):
        if verbose:
            print('Epoch {}/{}'.format(epoch, num_epochs - 1))
            print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in trainVal:
            running_correct = 0
            running_loss = 0
            running_total = 0
            if phase == 'train':
                imageLoader = train_loader
                model.train(True)
            else:
                imageLoader = validation_loader
                model.train(False)
                
            for data in imageLoader:
                image = data['x']
                label = data['y']
                if use_gpu:
                    image = image.type(torch.FloatTensor).cuda()
                    label = label.type(torch.FloatTensor).cuda()
                else:
                    image, label = inputs.type(torch.FloatTensor), label.type(torch.FloatTensor)
                optimizer.zero_grad()
                output = model(image)
                loss = criterion(output, label)
                num_imgs = image.size()[0]
                running_loss += loss.item()*num_imgs
                running_total += num_imgs
                if phase == 'train':
                    loss.backward()
                    optimizer.step()
            epoch_loss = float(running_loss)/float(running_total)
            #if verbose or (epoch%10 == 0):
            print('Phase:{}, epoch loss: {:.4f} '.format(phase, epoch_loss))
            loss_dict[phase].append(epoch_loss)
            if phase == 'val':
                if epoch_loss < best_loss:
                    best_loss = epoch_loss
                    best_model_wts = model.state_dict()
        if (epoch%10 == 0):
            time_elapsed = time.time() - since
            print('10 epoch costs {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val loss: {:4f}'.format(best_loss))    
    model.load_state_dict(best_model_wts)    
    return model, loss_dict

In [54]:
model_resnet = torchvision.models.resnet34(pretrained=True)
for param in model_resnet.parameters():
    param.requires_grad = False

# Parameters of newly constructed modules have requires_grad=True by default
num_ftrs = model_resnet.fc.in_features
model_resnet.fc = torch.nn.Linear(num_ftrs, 1)

In [55]:
use_gpu = torch.cuda.is_available()
if use_gpu:
    model = model_resnet.cuda()
     #This will optimize only the final layer since other layers the gradient calculation is removed and only parameters from the fc layer is given to the optimizer
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.001)

criterion = torch.nn.L1Loss()


In [56]:
best_model, loss_dict = train_model(model_resnet, criterion, optimizer,
                       num_epochs=40, trainVal=['train','val'])

Epoch 0/39
----------
Phase:train, epoch loss: 92.7554 
Phase:val, epoch loss: 61.4958 
10 epoch costs 3m 24s
Epoch 1/39
----------
Phase:train, epoch loss: 45.3378 
Phase:val, epoch loss: 36.8164 
Epoch 2/39
----------
Phase:train, epoch loss: 35.9019 
Phase:val, epoch loss: 34.6720 
Epoch 3/39
----------
Phase:train, epoch loss: 35.4179 
Phase:val, epoch loss: 35.2528 
Epoch 4/39
----------
Phase:train, epoch loss: 35.3335 
Phase:val, epoch loss: 34.0619 
Epoch 5/39
----------
Phase:train, epoch loss: 35.2779 
Phase:val, epoch loss: 34.0030 
Epoch 6/39
----------
Phase:train, epoch loss: 35.2723 
Phase:val, epoch loss: 33.7427 
Epoch 7/39
----------
Phase:train, epoch loss: 35.1210 
Phase:val, epoch loss: 33.8017 
Epoch 8/39
----------
Phase:train, epoch loss: 35.0755 
Phase:val, epoch loss: 33.8384 
Epoch 9/39
----------
Phase:train, epoch loss: 35.0454 
Phase:val, epoch loss: 33.6585 
Epoch 10/39
----------
Phase:train, epoch loss: 34.8662 
Phase:val, epoch loss: 33.4544 
10 epoch 